In [1]:
#!pip install sleap==1.2.6
import h5py

In [6]:
path = "displacement_analysis.h5"

def describe_hdf5(filename, attrs=True):
    """Utility to describe all items in an HDF5 file."""
    def desc(k, v):
        if type(v) == h5py.Dataset:
            print(f"[ds]  {v.name}: {v.shape} | dtype = {v.dtype}")
            if attrs and len(v.attrs) > 0:
                print(f"      attrs = {dict(v.attrs.items())}")
        elif type(v) == h5py.Group:
            print(f"[grp] {v.name}:")
            if attrs and len(v.attrs) > 0:
                print(f"      attrs = {dict(v.attrs.items())}")

    with h5py.File(filename, "r") as f:
        f.visititems(desc)
        
describe_hdf5(path)

[ds]  /edge_inds: (6, 2) | dtype = int64
[ds]  /edge_names: (6, 2) | dtype = |S9
[ds]  /instance_scores: (2, 54546) | dtype = float64
[ds]  /node_names: (6,) | dtype = |S9
[ds]  /point_scores: (2, 6, 54546) | dtype = float64
[ds]  /track_names: (2,) | dtype = |S7
[ds]  /track_occupancy: (54546, 2) | dtype = uint8
[ds]  /tracking_scores: (2, 54546) | dtype = float64
[ds]  /tracks: (2, 2, 6, 54546) | dtype = float64


In [7]:
with h5py.File(path) as f:
    tracks = f["tracks"][:]

tracks = np.transpose(tracks, [3, 0, 2, 1])
tracks.shape

NameError: name 'np' is not defined

In [ ]:
labels = sleap.load_file(path)
labels

In [ ]:
tracks = labels.numpy()
tracks.shape

In [ ]:
# Labeled frames can be accessed using [] indexing
labeled_frame = labels[100]
labeled_frame

In [ ]:
# Visualize the labeled frame
labeled_frame.plot(scale=0.5)

In [ ]:
# Instances from a labeled frame can be converted to numpy arrays together...
labeled_frame.numpy().shape

In [ ]:
# ... or individually:
predicted_instance = labeled_frame[0]
predicted_instance.numpy()

In [ ]:
# Trajectory across for all frames
plt.figure(figsize=(6, 6))
for j in range(tracks.shape[2]):
    plt.plot(tracks[:, 0, j, 0], tracks[:, 0, j, 1], "b-")
for j in range(tracks.shape[2]):
    plt.plot(tracks[:, 1, j, 0], tracks[:, 1, j, 1], "r-")
plt.grid(True);

In [ ]:
# Landmark-wise speed over time
speeds = np.linalg.norm(np.diff(tracks, axis=0), axis=3)  # (time, tracks, nodes)

plt.figure(figsize=(10, 4))
for j in range(tracks.shape[2]):
    plt.plot(speeds[:, 0, j], "b-")
for j in range(tracks.shape[2]):
    plt.plot(speeds[:, 1, j], "r-")
plt.xlabel("Time (frames)")
plt.ylabel("Landmark speed (px/frame)");